In [1]:
"""
Problem Defination: Given a sequence of chars, get the longest panlindorm it contains.

e.g 
Input: 'ABA'
Output: 'ABA'
Input: 'ABAAAAB'
Output: 'BAAAAB'
Input: 'I Live on the time, emit no evil at all.'
Output: 'Live on the time, emit no evil'
"""

"\nProblem Defination: Given a sequence of chars, get the longest panlindorm it contains.\n\ne.g \nInput: 'ABA'\nOutput: 'ABA'\nInput: 'ABAAAAB'\nOutput: 'BAAAAB'\nInput: 'I Live on the time, emit no evil at all.'\nOutput: 'Live on the time, emit no evil'\n"

In [7]:
import string

In [16]:
letters_digits = string.ascii_letters + string.digits

In [18]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [37]:
from icecream import ic

In [55]:
from functools import lru_cache

In [56]:
def is_panlindorm(string):
    string = ''.join([c for c in string if c in letters_digits]).upper()
    return string == string[::-1]

def get_prefix_panlindorm(string):
    candidates = [string[:i] for i in range(len(string)+1) if is_panlindorm(string[:i])]
    return max(candidates, key=len)

@lru_cache(maxsize=2*10)
def get_panlindorm(string):
    if len(string) == 1: return string
    return max([get_prefix_panlindorm(string), get_panlindorm(string[1:])], key=len).strip()

assert is_panlindorm('ABA')
assert is_panlindorm('ABBBA')
assert is_panlindorm('BBBB')
assert not is_panlindorm('ABAA')
assert is_panlindorm('Live on time, emit no evil')

assert get_prefix_panlindorm('ABA')  == 'ABA'
assert get_prefix_panlindorm('ABABBBB') == 'ABA'
assert get_prefix_panlindorm('ABCDEEEEEE') == 'A'

assert get_panlindorm('ABA') == 'ABA'
assert get_panlindorm('ABAAAAAA') == 'AAAAAA'
assert get_panlindorm('I live on time, emit no evil at all') == 'live on time, emit no evil'

## Knapsack

In [3]:
import random

In [6]:
from functools import lru_cache

In [7]:
items_random = [(random.randint(1, 10), random.randint(1, 20)) for _ in range(20)]
weights = [i[0] for i in items_random]
values = [i[1] for i in items_random]

In [331]:
K = 50

In [41]:
from icecream import ic

In [334]:
#solution = [0] * len(items_random)
solution = [[0] * len(items_random) for _ in range(K)]
@lru_cache(maxsize=2*10)
def knapsack(n, k):
    # n is the index of the last one candidates
    if n < 0: return 0
    if weights[n] > k:  return knapsack(n-1, k)
    
    exclude = knapsack(n-1, k)
    include = values[n] + knapsack(n - 1, k - weights[n])

    if include > exclude:
        solution[k][n] = 1
        return include
    else:
        solution[k][n] = 0
        return exclude

In [247]:
knapsack(len(items_random)-1, 1)

1 4


20

In [335]:
knapsack(len(items_random)-1, 20)

91

In [206]:
items_random

[(3, 10),
 (9, 20),
 (5, 7),
 (3, 9),
 (1, 20),
 (7, 10),
 (9, 8),
 (3, 4),
 (5, 15),
 (7, 8),
 (10, 14),
 (7, 5),
 (10, 13),
 (3, 10),
 (6, 8),
 (1, 19),
 (10, 1),
 (6, 17),
 (7, 8),
 (7, 7)]

In [210]:
solution[:5]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [117]:
solution[5][15]

1

In [119]:
weights[15]

1

In [125]:
solution[5 - 1][10]

0

对于任意的solution单元，例如 solution[i][j]， 其表示的是，当constraint时i的时候，最大的候选集是j的时候，j 选不选。 那么，如果j是1，就是 $item_j$被选择了，反之，没有被选择。 那么该如何判断选择了哪一个呢？ 

因为我们的递归过程是
```
exclude = knapsack(n-1, k)
include = values[n] + knapsack(n - 1, k - weights[n])

return max(exclude, include)
```
即，一直思考的是最后一个元素需不需要加进来，如果需要，就加上，如果加不进来，就不加。 所以，如果当前的solution[i][j]是0， 表示没有被选择，那么被选择的元素就是 soluion[i][j]， j 一直减小，遇到的第一个是 1 的元素。 

例如，对于 solution[4][19]是0，一路往前“推进”，第一个不是0的是15， 就是说，第15个item，是被选入的。 

当我们获得了一个解，那么我们的问题是，怎么获得接下来的解呢？ 
我们观察一下递归调用的过程： 
```
values[n] + knapsack(n - 1, k - weights[n])
```

就是说  **n** 有没有被选中，程序是依靠 **n-1**和 constraint 是 k - weights[n]确定的。 所以，当我们知道 constraint 是 **k**， **n**有没有被选择的时候，我们可以继续通过 **n-1**和 **k-weights[n]** 得知在 **n**和 **k**的时候，系统算出来的第2个答案。 然后以此类推。

所以，我们先定义一个函数，这个函数可以输入一个候选的item的最大序号n， 目前的constraint, 输出是这个情况下，系统选择的第1个item. 

In [133]:
solution[:5]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]

In [222]:
# it's a utility function
def first(iterables, predicate):
    iterables = iter(iterables)
    for i in iterables:
        if predicate(i): return i
    return None

def get_chosen_item(constraint, candidate_size, solve):
    index = first(enumerate(solve[constraint][:candidate_size][::-1]), predicate=lambda x: x[1] == 1)
    if index:
        return (candidate_size - 1) - index[0] 
    else:
        return None
    # when candidate_size set to 20, and the index is 0, means the last one is chosen, which index is 19
    # when candidate_size is 1, and the index is 0, means the item index 0, is selected.

In [328]:
get_chosen_item(constraint=4, candidate_size=20, solve=solution)

例如， 这个函数告诉我们，当 constraint是4，candidate_size是19的时候，16是必须要选择的。 

In [147]:
weights[15]

1

In [148]:
values[15]

19

我们在这个基础（constraint == 4, candidate_size=19）之上，继续获得 `knapsack(n - 1, k - weights[n])` 的解， 此时， 为了方便起见，因为我们已经知道了第一个被选择的是index == 15的item， 所以我们可以把candiadte_size 设置为 15, constraint == 4 - 1 (1是weights[15]的值), 那么我们能够继续获得下一个被选择的item

In [173]:
get_chosen_item(constraint=3, candidate_size=15, solve=solution)

4

基于此，我们递归获得完整的解决方案：

In [366]:
def get_complete_solution(solver, constraint, candidate_size=None):
    candidate_size = candidate_size or len(solver[0])
    index = get_chosen_item(constraint, candidate_size, solver)
    if index: 
        print('Get Item[{}], weight == {}, value == {}'.format(index, weights[index], values[index]))
        get_complete_solution(solver, constraint - weights[index], index)

In [223]:
solution

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [224]:
get_complete_solution(solution, constraint=1)

Get Item[4], weight == 1, value == 20


In [367]:
get_complete_solution(solution, constraint=4)

Get Item[15], weight == 1, value == 19
Get Item[4], weight == 1, value == 20


In [389]:
get_complete_solution(solution, constraint=19)

Get Item[17], weight == 6, value == 17
Get Item[15], weight == 1, value == 19
Get Item[13], weight == 3, value == 10
Get Item[8], weight == 5, value == 15
Get Item[4], weight == 1, value == 20


In [390]:
knapsack(k=19, n=len(items_random)-1)

91

## We Could Merge these together!

In [253]:
#solution = [0] * len(items_random)
# refactor it to with soltion in argument

@lru_cache(maxsize=2*10)
def knapsack(n, k, solution):
    # n is the index of the last one candidates
    if n < 0: return 0
    if weights[n] > k:  return knapsack(n-1, k, solution)
    
    exclude = knapsack(n-1, k, solution)
    include = values[n] + knapsack(n - 1, k - weights[n], solution)

    if include > exclude:
        solution[k][n] = 1
        print(k, n)
        return include
    else:
        solution[k][n] = 0
        return exclude

In [249]:
def knapsack_with_solution(items, constraint):
    weights = [w for w, v in items]
    values = [v for w, v in items]
    
    solution = [[0] * len(items) for _ in range(constraint+1)]
    
    optimal_result = knapsack(k=constraint, n=len(items)-1, solution=solution)
    print('Optimal Value Sum is : {}'.format(optimal_result))
    print(solution)
    get_complete_solution(solution, constraint=constraint)

In [250]:
knapsack_with_solution(items_random, 1)

TypeError: unhashable type: 'list'

In [254]:
# in order to let 'solution', the list type saved to cache, we need add an utility function

In [279]:
def to_list(tuple_data):
    return [to_list(t) if isinstance(t, tuple) else t for t in tuple_data]

def to_tuple(list_data):
    return tuple(to_tuple(t) if isinstance(t, list) else t for t in list_data)

In [288]:
from functools import wraps

In [310]:
def memo(func):
    cache = {}
    @wraps(func)
    def _wrap(*args, **kwargs):
        args = list(args)
        
        for ii, a in enumerate(args): 
            if isinstance(a, list): args[ii] = to_tuple(a)
        
        for k, v in kwargs.items():
            if isinstance(v, list): kwargs[k] = to_tuple(v)
                
        union_key = str(args) + str(kwargs)
        
        if union_key in cache: return cache[union_key]
        else:
            result = func(*args, **kwargs)
            cache[union_key] = result
            return result   
    return _wrap

In [311]:
def some_func_with_list(argument):
    print('called')
    return sum(argument)

In [312]:
for i in range(10):
    v = some_func_with_list([1, 2, 3])
    print(v)

called
6
called
6
called
6
called
6
called
6
called
6
called
6
called
6
called
6
called
6


In [313]:
# called 6 times!

In [314]:
@memo
def some_func_with_list(argument):
    print('called')
    return sum(argument)

In [315]:
for i in range(10):
    v = some_func_with_list([1, 2, 3])
    print(v)

called
6
6
6
6
6
6
6
6
6
6


In [319]:
# called 1 time, but print value 10 times.

In [307]:
# e.g 
to_list((1, 2, 3))

[1, 2, 3]

In [276]:
# e.g 
to_list((1, 2, 3, (1, 2, 3, (1, 2, 3))))

[1, 2, 3, [1, 2, 3, [1, 2, 3]]]

In [280]:
# e.g 
to_tuple([1, 2, 3])

(1, 2, 3)

In [281]:
# e.g
to_tuple([[1, 2, 3], [4, 5, 5]])

((1, 2, 3), (4, 5, 5))

In [320]:
#solution = [0] * len(items_random)
# refactor it to with soltion in argument

@memo
def knapsack(n, k, solution):
    # n is the index of the last one candidates
    if n < 0: return 0
    if weights[n] > k:  return knapsack(n-1, k, solution)
    
    exclude = knapsack(n-1, k, solution)
    include = values[n] + knapsack(n - 1, k - weights[n], solution)

    if include > exclude:
        solution[k][n] = 1
        print(k, n)
        return include
    else:
        solution[k][n] = 0
        return exclude

## Optimizatoin

## Dynamic Programming Optimization

## Linear Optimization

## Constraint Optimization Programming

## Max Intergral Optimization

In [217]:
items_random

[(7, 16),
 (2, 14),
 (2, 6),
 (2, 5),
 (3, 16),
 (2, 17),
 (4, 6),
 (7, 5),
 (10, 8),
 (1, 13),
 (3, 10),
 (10, 12),
 (3, 19),
 (7, 12),
 (5, 4),
 (8, 14),
 (10, 12),
 (4, 10),
 (2, 20),
 (3, 15)]

In [222]:
kk

[0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

33

In [116]:
items_random

[(7, 16),
 (2, 14),
 (2, 6),
 (2, 5),
 (3, 16),
 (2, 17),
 (4, 6),
 (7, 5),
 (10, 8),
 (1, 13),
 (3, 10),
 (10, 12),
 (3, 19),
 (7, 12),
 (5, 4),
 (8, 14),
 (10, 12),
 (4, 10),
 (2, 20),
 (3, 15)]

In [103]:
weights

[7, 2, 2, 2, 3, 2, 4, 7, 10, 1, 3, 10, 3, 7, 5, 8, 10, 4, 2, 3]

In [104]:
values

[16, 14, 6, 5, 16, 17, 6, 5, 8, 13, 10, 12, 19, 12, 4, 14, 12, 10, 20, 15]

In [93]:
len(weights)

20

In [102]:
knapsack(19, 1)

13

## CV 
+ 可以参考 Stanford NLP, CV Group 
+ 素雅 
+ 突出重点

## 面试 
+ 突出自己的优点, e.g 学习能力 我的抗压能力
+ 引导面试官
+ 优秀的面试官发你的潜力 发现的你长处 看看你这个长处到底有多长
+ 糟糕的面试官是，只关注你会不会他想让你知道的那个东西，关注你的“经历”

## 快速入行 技能驱动 
## 不管你在哪个公司 只要面对的问题大体类似 